# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [3]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [4]:
!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [6]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [7]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2024-10-01 09:20:19--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0s      

2024-10-01 09:20:19 (137 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2024-10-01 09:20:19--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sav

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [9]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 28, 9, 20, 20, 837118)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [10]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [11]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [12]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [13]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [14]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [15]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to batch ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Invalid token"}')
post: trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=0a30219e-7db0-4576-a26b-3442f53b3018; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=61520699-692a-4e2f-ad43-34ec3465c9a8; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=41e157d9-335d-4f8d-9653-331d44658455; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=06f2c714-d710-4e4a-b5d9-5974a3dc63be; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=7e0a75e5-5524-4ebf-87a7-fd48982643cf; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=ff60ee62-248b-4b1e-a16a-32dd95857fa9
Failed to batch ingest runs: langsmith.utils.L

'Yes, people generally liked John Wick based on the reviews provided.'

In [16]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

Failed to batch ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Invalid token"}')
post: trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=e3557d82-40b6-4e46-85c0-904ef0ef3739; trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=40eab55e-c2fc-4a9c-b01f-819f3514a092; trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=9959a2f3-f203-4ae2-a44e-964683a609f1; trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=6381ca9b-8f90-4d91-a43d-fae3b4af26a5; trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=c9ba6e74-6d19-4a23-b299-27189058e961; trace=e3557d82-40b6-4e46-85c0-904ef0ef3739,id=8ff78c87-15c8-4bef-8640-d06e5eb39286; patch: trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=080b19c3-d6d7-425d-8d32-749f18815cd0; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=8d5d0cc0-936d-488d-b91f-6201f12d8419; trace=0a30219e-7db0-4576-a26b-3442f53b3018,id=0a30219e

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [17]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," the story follows an ex-hit-man who comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The film is filled with violent action, shootouts, and breathtaking fights as John Wick unleashes a maelstrom of destruction against those who come after him, leading to a relentless vendetta.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [18]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [19]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [20]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, opinions on the John Wick movies seem to vary. Some people really enjoyed the first movie, praising its action sequences and style, while others found the third installment to be lacking in plot and excessively violent. Overall, it seems that John Wick has a mix of positive and negative reviews.'

In [21]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry, there are no reviews with a rating of 10 in the provided context."

In [22]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist is a former hitman seeking vengeance for the death of his dog, which was a final gift from his deceased wife. The movie is known for its intense action sequences and choreography.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [23]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [24]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided.'

In [26]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review Title: A Masterpiece & Brilliant Sequel\n- URL: '/review/rw4854296/?ref_=tt_urv'"

In [27]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, John Wick, is forced out of retirement by a mobster who blows up his house. Wick is then given a task to kill the mobster's sister in Rome, but after completing it, a contract is put on his life. Wick then seeks revenge on the mobster who betrayed him."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [28]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [29]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [30]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People generally liked John Wick based on the reviews provided. The film was praised for its slickness, brilliant action sequences, fast-paced nature, and Keanu Reeves' performance. Reviewers found it to be a refreshing and entertaining action film."

In [31]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I don't know."

In [32]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, played by Keanu Reeves, is a retired assassin who comes out of retirement when someone kills his dog. This leads to a series of events involving revenge, car theft, and a lot of violence. In the sequel, John Wick is called on to pay off an old debt by helping take over the Assassin's Guild, leading to him flying around to different locations and killing many assassins."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [33]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [34]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/kd/n70wp5t13t1ftd6d335l_ynm0000gn/T/ipykernel_30206/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [35]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [36]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [37]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [38]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People's opinions on John Wick seem to vary. Some love the series and find it consistent and well-received, while others have strong negative opinions about it. It ultimately depends on individual preferences."

In [39]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review: "/review/rw4854296/?ref_=tt_urv" for the movie "John Wick 3".'

In [40]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, played by Keanu Reeves, is a retired assassin who comes out of retirement to seek revenge when someone kills his dog and steals his car. He gets caught up in a series of events involving the Assassin's Guild and goes on a killing spree across Italy, Canada, and Manhattan."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [41]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [42]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [43]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. The action sequences, Keanu Reeves' performance, and the overall style of the film were praised by many reviewers."

In [44]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [45]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story is filled with violent action, shootouts, and breathtaking fights as John Wick unleashes a maelstrom of destruction against those who come after him.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [46]:
!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [47]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [48]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [49]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [50]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [51]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [52]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews and high ratings given by viewers.'

In [53]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [54]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The movie is filled with violent action, shootouts, and breathtaking fights as John Wick unleashes a maelstrom of destruction against those who come after him.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

### YOUR CODE HERE

#### Set up Langsmith

In [59]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Advanced_Retrieval_with_Langchain - {unique_id}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

#### Generate the synthetic dataset using the SDG.ipynb notebook

In [60]:
import pandas as pd 

synthetic_testset_df = pd.read_csv("synthetic_testset.csv")
synthetic_testset_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,What is the reputation of the John Wick action...,"["": 21\nReview: He fights and bites, he fights...",The John Wick action movie franchise is known ...,simple,"[{'source': 'john_wick_4.csv', 'row': 21, 'Rev...",True
1,1,What role does Iosef's uncle play in the plot ...,"["": 5\nReview: Iosef's uncle still has John Wi...",Iosef's uncle plays a role in the plot of John...,simple,"[{'source': 'john_wick_2.csv', 'row': 5, 'Revi...",True
2,2,How does John Wick: Chapter 3 - Parabellum exp...,"["": 24\nReview: John Wick: Chapter 3 - Parabel...",John Wick: Chapter 3 - Parabellum explores the...,simple,"[{'source': 'john_wick_3.csv', 'row': 24, 'Rev...",True
3,3,What role does the Continental hotel play in t...,"["": 20\nReview: After resolving his issues wit...","The Continental hotel in New York City, owned ...",simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
4,4,What is the premise of the John Wick movie ser...,"["": 19\nReview: If you've seen the first John ...",The premise of the John Wick movie series revo...,simple,"[{'source': 'john_wick_2.csv', 'row': 19, 'Rev...",True
5,5,Who is the mobster that visits John Wick and a...,"["": 20\nReview: After resolving his issues wit...",Santino D'Antonio is the mobster who visits Jo...,simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
6,6,What aspect of 'John Wick' received praise for...,[': 19\nReview: I really don\'t understand the...,The cinematography and fight choreography of '...,simple,"[{'source': 'john_wick_1.csv', 'row': 19, 'Rev...",True
7,7,What role does the marker play in the plot of ...,"["": 20\nReview: After resolving his issues wit...",The marker in the plot of John Wick: Chapter 2...,simple,"[{'source': 'john_wick_2.csv', 'row': 20, 'Rev...",True
8,8,What event in the movie John Wick Chapter 2 le...,"["": 10\nReview: Wow what a great surprise this...",The answer to given question is not present in...,simple,"[{'source': 'john_wick_1.csv', 'row': 10, 'Rev...",True
9,9,What role do assassins play in the artistic sp...,"["": 12\nReview: If there's an equivalent of cl...",Assassins play a significant role in the artis...,simple,"[{'source': 'john_wick_2.csv', 'row': 12, 'Rev...",True


# Create a function to test each chaing using the synthetic testset

In [61]:
from datasets import Dataset
import pandas as pd 

def test_chain(testset_path: str, chain) -> Dataset:
    
    testset_df = pd.read_csv(testset_path)

    answers = []
    contexts = []

    questions = testset_df["question"].values.tolist()
    groundtruths = testset_df["ground_truth"].values.tolist()

    # Loop over the list of synthetic questions and call the chain 
    for question in questions:
        print(question)

        # Call the chain
        response = chain.invoke({"question" : question})
        print(response)
        
        # Capture the answer and context 
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])

    data = {
        "question" : questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : groundtruths
        }

    response_ds = Dataset.from_dict(data)

    return response_ds

In [62]:
naive_chain_ds = test_chain("synthetic_testset.csv", naive_retrieval_chain)
naive_chain_ds.to_csv("datasets/naive_chain_ds.csv")
naive_chain_ds.info.description = "naive_chain_ds"

What is the reputation of the John Wick action movie franchise in terms of its overall tone and style?
{'response': AIMessage(content='The John Wick action movie franchise is highly regarded for its overall tone and style. It is described as slick, violent fun with stylish stunts, kinetic chaos, and cool characters. The action sequences are praised for being expertly choreographed, shot, and edited, setting a high standard for action filmmaking. Overall, the franchise is known for delivering entertaining and relentless action that keeps viewers engaged.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 3554, 'total_tokens': 3633, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a482752e-51bf-4589-8384-3847e4c0fd77-0', usage_metadata={'input_tokens': 3554, 'output_tokens': 79, 'total_tokens': 3633}), '

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
bm25_chain_ds = test_chain("synthetic_testset.csv", bm25_retrieval_chain)
bm25_chain_ds.to_csv("datasets/bm25_chain_ds.csv")
bm25_chain_ds.info.description = "bm25_chain_ds"

In [ ]:
contextual_compression_chain_ds = test_chain("synthetic_testset.csv", contextual_compression_retrieval_chain)
contextual_compression_chain_ds.to_csv("datasets/contextual_compression_chain_ds.csv")
contextual_compression_chain_ds.info.description = "contextual_compression_chain_ds"

In [ ]:
multi_query_chain_ds = test_chain("synthetic_testset.csv", multi_query_retrieval_chain)
multi_query_chain_ds.to_csv("datasets/multi_query_chain_ds.csv")
multi_query_chain_ds.info.description = "multi_query_chain_ds"

In [ ]:
parent_document_chain_ds = test_chain("synthetic_testset.csv", parent_document_retrieval_chain)
parent_document_chain_ds.to_csv("datasets/parent_document_chain_ds.csv")
parent_document_chain_ds.info.description = "parent_document_chain_ds"

In [ ]:
ensemble_chain_ds = test_chain("synthetic_testset.csv", ensemble_retrieval_chain)
ensemble_chain_ds.to_csv("datasets/ensemble_chain_ds.csv")
ensemble_chain_ds.info.description = "ensemble_chain_ds"

In [ ]:
semantic_chain_ds = test_chain("synthetic_testset.csv", semantic_retrieval_chain)
semantic_chain_ds.to_csv("datasets/semantic_chain_ds.csv")
semantic_chain_ds.info.description = "semantic_chain_ds"

#### Run Ragas evaluations on the retrieval chain datasets using metrics focused on retrieval

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    context_recall,             # Context recall measures if all the relevant information required to answer the question was retrieved. This metric is computed based on the ground_truth 
    context_precision,          # Context precision measures the signal-to-noise ratio of the retrieved context. This metric is computed using the question and the contexts.
)

metrics = [
    context_recall,
    context_precision,
]

In [69]:
datasets = [naive_chain_ds,
         bm25_chain_ds,
         contextual_compression_chain_ds,
         multi_query_chain_ds,
         parent_document_chain_ds,
         ensemble_chain_ds,
         semantic_chain_ds,
         ]

In [ ]:
ragas_results = []

for dataset in datasets:
    dataset_name = dataset.info.description
    print(dataset_name)
    
    ragas_result = evaluate(dataset, metrics)

    ragas_results_df = ragas_result.to_pandas() 
    
    ragas_results_df.to_csv(f"ragas/{dataset_name}_ragas_results.csv")
    
    ragas_results.append(ragas_result)



In [ ]:
for ragas_result in ragas_results:
    print(ragas_result)